In [151]:
!pip install numpy
!pip install scikit-learn


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [248]:
from pyspark.sql import SparkSession
import numpy as np
from sklearn.metrics import confusion_matrix

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StandardScaler

from pyspark.ml.stat import Correlation
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [249]:
spark = (
    SparkSession
    .builder
    .appName("IRIS")
    .master("local[*]")
    .getOrCreate()
)

spark

In [250]:
df = spark.read.csv('data/Iris.csv', header=True, inferSchema=True)

In [251]:
df.show()

+---+-------------+------------+-------------+------------+-----------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|
+---+-------------+------------+-------------+------------+-----------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|
|  5|          5.0|         3.6|          1.4|         0.2|Iris-setosa|
|  6|          5.4|         3.9|          1.7|         0.4|Iris-setosa|
|  7|          4.6|         3.4|          1.4|         0.3|Iris-setosa|
|  8|          5.0|         3.4|          1.5|         0.2|Iris-setosa|
|  9|          4.4|         2.9|          1.4|         0.2|Iris-setosa|
| 10|          4.9|         3.1|          1.5|         0.1|Iris-setosa|
| 11|          5.4|         3.7|          1.5|         0.2|Iris-

In [252]:
print(df.count())

150


In [253]:
df = df.drop('Id')

In [254]:
df.printSchema()

root
 |-- SepalLengthCm: double (nullable = true)
 |-- SepalWidthCm: double (nullable = true)
 |-- PetalLengthCm: double (nullable = true)
 |-- PetalWidthCm: double (nullable = true)
 |-- Species: string (nullable = true)



In [255]:
df.describe().show()

+-------+------------------+-------------------+------------------+------------------+--------------+
|summary|     SepalLengthCm|       SepalWidthCm|     PetalLengthCm|      PetalWidthCm|       Species|
+-------+------------------+-------------------+------------------+------------------+--------------+
|  count|               150|                150|               150|               150|           150|
|   mean| 5.843333333333335| 3.0540000000000007|3.7586666666666693|1.1986666666666672|          null|
| stddev|0.8280661279778637|0.43359431136217375| 1.764420419952262|0.7631607417008414|          null|
|    min|               4.3|                2.0|               1.0|               0.1|   Iris-setosa|
|    max|               7.9|                4.4|               6.9|               2.5|Iris-virginica|
+-------+------------------+-------------------+------------------+------------------+--------------+



In [256]:
df.groupBy('Species').count().show()

+---------------+-----+
|        Species|count|
+---------------+-----+
| Iris-virginica|   50|
|    Iris-setosa|   50|
|Iris-versicolor|   50|
+---------------+-----+



In [257]:
# Show Duplicates
df.groupBy(df.columns).count().filter('count>1').show(truncate=False)

+-------------+------------+-------------+------------+--------------+-----+
|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|Species       |count|
+-------------+------------+-------------+------------+--------------+-----+
|4.9          |3.1         |1.5          |0.1         |Iris-setosa   |3    |
|5.8          |2.7         |5.1          |1.9         |Iris-virginica|2    |
+-------------+------------+-------------+------------+--------------+-----+



In [258]:
df = df.dropDuplicates()

In [259]:
train_df, test_df = df.randomSplit([0.8, 0.2],seed=42)

In [260]:
feat = ['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']

# Coding the label
indexer = StringIndexer(inputCol="Species", outputCol='Label',handleInvalid="keep")

# Single vector features
assembler = VectorAssembler(inputCols=feat, outputCol='features')

# 
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withMean=False, withStd=True)

pipeline = Pipeline(stages=[indexer,assembler,scaler])
pipeline_model = pipeline.fit(train_df)

train = pipeline_model.transform(train_df)
test = pipeline_model.transform(test_df)

In [261]:
train.show()

+-------------+------------+-------------+------------+---------------+-----+-----------------+--------------------+
|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|        Species|Label|         features|     scaled_features|
+-------------+------------+-------------+------------+---------------+-----+-----------------+--------------------+
|          4.3|         3.0|          1.1|         0.1|    Iris-setosa|  2.0|[4.3,3.0,1.1,0.1]|[5.13903783422384...|
|          4.4|         2.9|          1.4|         0.2|    Iris-setosa|  2.0|[4.4,2.9,1.4,0.2]|[5.25855034199649...|
|          4.4|         3.2|          1.3|         0.2|    Iris-setosa|  2.0|[4.4,3.2,1.3,0.2]|[5.25855034199649...|
|          4.5|         2.3|          1.3|         0.3|    Iris-setosa|  2.0|[4.5,2.3,1.3,0.3]|[5.37806284976913...|
|          4.6|         3.1|          1.5|         0.2|    Iris-setosa|  2.0|[4.6,3.1,1.5,0.2]|[5.49757535754178...|
|          4.6|         3.4|          1.4|         0.3|    Iris-

In [262]:
# Show Correlation Matrix
train_vector = train.select('features')

matrix = Correlation.corr(train_vector, 'features').head()[0]

print("Matriz de correlação:\n" + str(matrix.toArray()))

Matriz de correlação:
[[ 1.         -0.05390719  0.87040641  0.80438877]
 [-0.05390719  1.         -0.36550151 -0.30400807]
 [ 0.87040641 -0.36550151  1.          0.95732899]
 [ 0.80438877 -0.30400807  0.95732899  1.        ]]


In [263]:
train = train.select('scaled_features', 'Label')
test = test.select('scaled_features', 'Label')

In [264]:
train.show()

+--------------------+-----+
|     scaled_features|Label|
+--------------------+-----+
|[5.13903783422384...|  2.0|
|[5.25855034199649...|  2.0|
|[5.25855034199649...|  2.0|
|[5.37806284976913...|  2.0|
|[5.49757535754178...|  2.0|
|[5.49757535754178...|  2.0|
|[5.61708786531443...|  2.0|
|[5.61708786531443...|  2.0|
|[5.73660037308708...|  2.0|
|[5.73660037308708...|  2.0|
|[5.73660037308708...|  2.0|
|[5.73660037308708...|  2.0|
|[5.85611288085973...|  1.0|
|[5.85611288085973...|  0.0|
|[5.85611288085973...|  2.0|
|[5.97562538863237...|  1.0|
|[5.97562538863237...|  1.0|
|[5.97562538863237...|  2.0|
|[5.97562538863237...|  2.0|
|[5.97562538863237...|  2.0|
+--------------------+-----+
only showing top 20 rows



In [265]:
classifiers = [
    ("NaiveBayes", NaiveBayes(featuresCol='scaled_features', labelCol='Label')),
    ("LogisticRegression", LogisticRegression(featuresCol='scaled_features', labelCol='Label')),
    ("DecisionTree", DecisionTreeClassifier(featuresCol='scaled_features', labelCol='Label'))
]

evaluator = MulticlassClassificationEvaluator(labelCol='Label', predictionCol='prediction')

In [266]:
for name, classifier in classifiers:
    model = classifier.fit(train)

    predictions = model.transform(test)

    accuracy = evaluator.evaluate(predictions)

    print(f"Precisão do {name}: {accuracy}")
    print("--------------------------------------------------")


Precisão do NaiveBayes: 0.9569597069597069
--------------------------------------------------
Precisão do LogisticRegression: 0.9569597069597069
--------------------------------------------------
Precisão do DecisionTree: 1.0
--------------------------------------------------


In [267]:
predictions.show()

+--------------------+-----+------------------+-----------------+----------+
|     scaled_features|Label|     rawPrediction|      probability|prediction|
+--------------------+-----+------------------+-----------------+----------+
|[5.25855034199649...|  2.0|[0.0,0.0,34.0,0.0]|[0.0,0.0,1.0,0.0]|       2.0|
|[5.49757535754178...|  2.0|[0.0,0.0,34.0,0.0]|[0.0,0.0,1.0,0.0]|       2.0|
|[5.49757535754178...|  2.0|[0.0,0.0,34.0,0.0]|[0.0,0.0,1.0,0.0]|       2.0|
|[5.73660037308708...|  2.0|[0.0,0.0,34.0,0.0]|[0.0,0.0,1.0,0.0]|       2.0|
|[5.85611288085973...|  2.0|[0.0,0.0,34.0,0.0]|[0.0,0.0,1.0,0.0]|       2.0|
|[5.97562538863237...|  2.0|[0.0,0.0,34.0,0.0]|[0.0,0.0,1.0,0.0]|       2.0|
|[5.97562538863237...|  2.0|[0.0,0.0,34.0,0.0]|[0.0,0.0,1.0,0.0]|       2.0|
|[6.09513789640502...|  2.0|[0.0,0.0,34.0,0.0]|[0.0,0.0,1.0,0.0]|       2.0|
|[6.45367541972296...|  2.0|[0.0,0.0,34.0,0.0]|[0.0,0.0,1.0,0.0]|       2.0|
|[6.45367541972296...|  2.0|[0.0,0.0,34.0,0.0]|[0.0,0.0,1.0,0.0]|       2.0|

In [268]:
# confusion matrix
predictions.groupBy('Label', 'prediction').count().show()

+-----+----------+-----+
|Label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|    6|
|  2.0|       2.0|   14|
|  0.0|       0.0|    4|
+-----+----------+-----+



In [130]:
spark.stop()